<a href="https://colab.research.google.com/github/Amirno/opencv/blob/master/Another_copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate,Flatten,Dense,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import to_categorical

In [ ]:
import struct
import numpy as np
import cv2


def __convert_to_one_hot(vector, num_classes):
    result = np.zeros(shape=[len(vector), num_classes])
    result[np.arange(len(vector)), vector] = 1
    return result


def __resize_image(src_image, dst_image_height, dst_image_width):
    src_image_height = src_image.shape[0]
    src_image_width = src_image.shape[1]

    if src_image_height > dst_image_height or src_image_width > dst_image_width:
        height_scale = dst_image_height / src_image_height
        width_scale = dst_image_width / src_image_width
        scale = min(height_scale, width_scale)
        img = cv2.resize(src=src_image, dsize=(0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    else:
        img = src_image

    img_height = img.shape[0]
    img_width = img.shape[1]

    dst_image = np.zeros(shape=[dst_image_height, dst_image_width], dtype=np.uint8)

    y_offset = (dst_image_height - img_height) // 2
    x_offset = (dst_image_width - img_width) // 2

    dst_image[y_offset:y_offset+img_height, x_offset:x_offset+img_width] = img

    return dst_image


def read_hoda_cdb(file_name):
    with open(file_name, 'rb') as binary_file:

        data = binary_file.read()

        offset = 0

        # read private header

        yy = struct.unpack_from('H', data, offset)[0]
        offset += 2

        m = struct.unpack_from('B', data, offset)[0]
        offset += 1

        d = struct.unpack_from('B', data, offset)[0]
        offset += 1

        H = struct.unpack_from('B', data, offset)[0]
        offset += 1

        W = struct.unpack_from('B', data, offset)[0]
        offset += 1

        TotalRec = struct.unpack_from('I', data, offset)[0]
        offset += 4

        LetterCount = struct.unpack_from('128I', data, offset)
        offset += 128 * 4

        imgType = struct.unpack_from('B', data, offset)[0]  # 0: binary, 1: gray
        offset += 1

        Comments = struct.unpack_from('256c', data, offset)
        offset += 256 * 1

        Reserved = struct.unpack_from('245c', data, offset)
        offset += 245 * 1

        if (W > 0) and (H > 0):
            normal = True
        else:
            normal = False

        images = []
        labels = []

        for i in range(TotalRec):

            StartByte = struct.unpack_from('B', data, offset)[0]  # must be 0xff
            offset += 1

            label = struct.unpack_from('B', data, offset)[0]
            offset += 1

            if not normal:
                W = struct.unpack_from('B', data, offset)[0]
                offset += 1

                H = struct.unpack_from('B', data, offset)[0]
                offset += 1

            ByteCount = struct.unpack_from('H', data, offset)[0]
            offset += 2

            image = np.zeros(shape=[H, W], dtype=np.uint8)

            if imgType == 0:
                # Binary
                for y in range(H):
                    bWhite = True
                    counter = 0
                    while counter < W:
                        WBcount = struct.unpack_from('B', data, offset)[0]
                        offset += 1
                        # x = 0
                        # while x < WBcount:
                        #     if bWhite:
                        #         image[y, x + counter] = 0  # Background
                        #     else:
                        #         image[y, x + counter] = 255  # ForeGround
                        #     x += 1
                        if bWhite:
                            image[y, counter:counter + WBcount] = 0  # Background
                        else:
                            image[y, counter:counter + WBcount] = 255  # ForeGround
                        bWhite = not bWhite  # black white black white ...
                        counter += WBcount
            else:
                # GrayScale mode
                data = struct.unpack_from('{}B'.format(W * H), data, offset)
                offset += W * H
                image = np.asarray(data, dtype=np.uint8).reshape([W, H]).T

            images.append(image)
            labels.append(label)

        return images, labels


def read_hoda_dataset(dataset_path, images_height=32, images_width=32, one_hot=False, reshape=True):
    images, labels = read_hoda_cdb(dataset_path)
    assert len(images) == len(labels)

    X = np.zeros(shape=[len(images), images_height, images_width], dtype=np.float32)
    Y = np.zeros(shape=[len(labels)], dtype=np.int64)

    for i in range(len(images)):
        image = images[i]
        # Image resizing.
        image = __resize_image(src_image=image, dst_image_height=images_height, dst_image_width=images_width)
        # Image normalization.
        image = image / 255
        # Image binarization.
        image = np.where(image >= 0.5, 1, 0)
        # Image.
        X[i] = image
        # Label.
        Y[i] = labels[i]

    if one_hot:
        Y = __convert_to_one_hot(Y, 10).astype(dtype=np.float32)
    else:
        Y = Y.astype(dtype=np.float32)

    if reshape:
        X = X.reshape(-1, images_height * images_width)
    else:
        X = X.reshape(-1, images_height, images_width, 1)

    return X, Y


In [ ]:
# Define U-Net model architecture
def unet_model_b(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Contracting path
    conv1 = layers.Conv2D(32, 3, activation="relu", padding="same")(inputs)
    conv1 = layers.Conv2D(32, 3, activation="relu", padding="same")(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(64, 3, activation="relu", padding="same")(pool1)
    conv2 = layers.Conv2D(64, 3, activation="relu", padding="same")(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(128, 3, activation="relu", padding="same")(pool2)
    conv3 = layers.Conv2D(128, 3, activation="relu", padding="same")(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(256, 3, activation="relu", padding="same")(pool3)
    conv4 = layers.Conv2D(256, 3, activation="relu", padding="same")(conv4)
    drop4 = layers.Dropout(0.5)(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bottom
    conv5 = layers.Conv2D(512, 3, activation="relu", padding="same")(pool4)
    conv5 = layers.Conv2D(512, 3, activation="relu", padding="same")(conv5)
    drop5 = layers.Dropout(0.5)(conv5)

    # Expansive path
    up6 = layers.Conv2D(256, 2, activation="relu", padding="same")(layers.UpSampling2D(size=(2, 2))(drop5))
    merge6 = layers.concatenate([drop4, up6], axis=3)
    conv6 = layers.Conv2D(256, 3, activation="relu", padding="same")(merge6)
    conv6 = layers.Conv2D(256, 3, activation="relu", padding="same")(conv6)

    up7 = layers.Conv2D(128, 2, activation="relu", padding="same")(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.concatenate([conv3, up7], axis=3)
    conv7 = layers.Conv2D(128, 3, activation="relu", padding="same")(merge7)
    conv7 = layers.Conv2D(128, 3, activation="relu", padding="same")(conv7)

    up8 = layers.Conv2D(64, 2, activation="relu", padding="same")(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.concatenate([conv2, up8], axis=3)
    conv8 = layers.Conv2D(64, 3, activation="relu", padding="same")(merge8)
    conv8 = layers.Conv2D(64, 3, activation="relu", padding="same")(conv8)

    up9 = layers.Conv2D(32, 2, activation="relu", padding="same")(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.concatenate([conv1, up9], axis=3)
    conv9 = layers.Conv2D(32, 3, activation="relu", padding="same")(merge9)
    conv9 = layers.Conv2D(32, 3, activation="relu", padding="same")(conv9)
    conv9 = layers.Conv2D(1, 3, activation="relu", padding="same")(conv9)
    conv9 = layers.Flatten()(conv9)
    conv9 = layers.Dense(100, activation="relu")(conv9)
    #output = layers.Conv2D(10, (1, 1), activation='softmax')(conv9)
    conv9 = layers.Dense(num_classes, activation="softmax")(conv9)

    model = tf.keras.Model(inputs=inputs, outputs=conv9)
    return model


In [ ]:
def augmanet(x_train,y_train):
# Define the augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1
        )

# Fit the data generator on the training data
    datagen.fit(x_train)

    # Generate augmented images
    augmented_images = []
    augmented_labels = []

    for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=50000, shuffle=True):
        augmented_images.append(x_batch)
        augmented_labels.append(y_batch)
        break

    # Convert augmented images and labels to arrays
    augmented_images = np.array(augmented_images).reshape(-1, 32, 32, 1)
    augmented_labels = np.array(augmented_labels).reshape(-1)

    # Concatenate augmented data with original data
    x_train_augmented = np.concatenate((x_train, augmented_images))
    y_train_augmented = np.concatenate((y_train, augmented_labels))
    return x_train_augmented,y_train_augmented

In [ ]:
def unet_new(input_shape):
    inputs = Input(shape=input_shape)

    # Encoding path
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

    # Decoding path
    up4 = UpSampling2D(size=(2, 2))(conv3)
    up4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up4)
    merge4 = concatenate([conv2, up4], axis=3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    up5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up5)
    merge5 = concatenate([conv1, up5], axis=3)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(merge5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = Conv2D(8, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = Conv2D(1, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = Flatten()(conv5)
    conv5 = Dense(400, activation='relu')(conv5)
    conv5 = Dense(100, activation='relu')(conv5)
    conv5=Dropout(0.4)(conv5)
    # Output
    output = Dense(10, activation='softmax')(conv5)

    return Model(inputs, output)

In [ ]:
def unet(input_shape):
    inputs = Input(shape=input_shape)

    # Encoding path
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

    # Decoding path
    up4 = UpSampling2D(size=(2, 2))(conv3)
    up4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up4)
    merge4 = concatenate([conv2, up4], axis=3)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge4)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

    up5 = UpSampling2D(size=(2, 2))(conv4)
    up5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up5)
    merge5 = concatenate([conv1, up5], axis=3)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(merge5)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = Conv2D(1, (5, 5), activation='relu', padding='same')(conv5)
    conv5=Dropout(0.4)(conv5)
    conv5 = Flatten()(conv5)
    conv5 = Dense(100, activation='relu')(conv5)
    # Output
    output = Dense(10, activation='softmax')(conv5)

    return Model(inputs, output)

In [ ]:
X_train, Y_train = read_hoda_dataset(dataset_path='./HDR/Train 60000.cdb',
                                images_height=32,
                                images_width=32,
                                one_hot=False,
                                reshape=False)
print('Reading test dataset (Test 20000.cdb)...')
X_test, Y_test = read_hoda_dataset(dataset_path='./HDR/Test 20000.cdb',
                              images_height=32,
                              images_width=32,
                              one_hot=False,
                              reshape=False)

print('Reading remaining samples dataset (RemainingSamples.cdb)...')
X_remain, Y_remain = read_hoda_dataset('./HDR/RemainingSamples.cdb',
                                             images_height=32,
                                             images_width=32,
                                             one_hot=False,
                                             reshape=False)



Reading test dataset (Test 20000.cdb)...
Reading remaining samples dataset (RemainingSamples.cdb)...


In [ ]:
[X_train,Y_train]=augmanet(X_train,Y_train)

In [ ]:
X_remain=X_remain.reshape((22352,32,32))
X_train=X_train.reshape((110000,32,32))
    # X_train=X_train.astype('float32')/255
X_test=X_test.reshape((20000,32,32))
    # X_test=X_test.astype('float32')/255


In [ ]:
Y_train=to_categorical(Y_train)
Y_test=to_categorical(Y_test)
Y_remain=to_categorical(Y_remain)
input_shape = (32, 32, 1)
num_classes = 10

In [ ]:

# Create the U-Net model
#model = unet_model_b(input_shape, num_classes)
model = unet_new(input_shape)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


# Train the model
# Assuming you have your training data and labels as X_train and y_train
#model.fit(X_train, Y_train, batch_size=256, epochs=1,validation_data=(X_test,Y_test))
h=model.fit(X_train, Y_train, batch_size=256, epochs=10,validation_data=(X_remain,Y_remain))
#model.fit(X_train, Y_train, batch_size=128, epochs=10,validation_split=0.2)
#99.34

#Apologies for the incomplete code. Here's the complete code snippet for training the U-Net model for Persian handwritten digit recognition:

loss, accuracy = model.evaluate(X_test, Y_test)

# Make predictions on new data
# Assuming you have new data as X_new
predictions = model.predict(X_test)

Epoch 1/10
  3/430 ━━━━━━━━━━━━━━━━━━━━ 58:41 8s/step - accuracy: 0.1115 - loss: 2.2996